In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<!-- <table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/notebook_template.ipynb"">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/notebook_template.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table> -->

# Orchestrating a workflow to train and deploy a R model using [Vertex Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)

## Overview

This tutorial uses R.A. Fisher's Iris dataset, a small dataset that is popular for trying out machine learning techniques. Each instance has four numerical features, which are different measurements of a flower, and a target label that marks it as one of three types of iris: Iris setosa, Iris versicolour, or Iris virginica. Flower classifier with an e2e example using [Vertex Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines).

This `kfp.v2` code currently does not run on OSS KFP, but will be able to soon once the 'v2 compatibility mode' feature is supported.


### Set up your local development environment

**If you are using Colab or Google Cloud Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages



On colab, authenticate first:

In [ ]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

Then, install the libraries.

In [1]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

We will be using [Vertex SDK for Python](https://cloud.google.com/vertex-ai/docs/start/client-libraries#python) to interact with Vertex AI services. The high-level aiplatform library is designed to simplify common data science workflows by using wrapper classes and opinionated defaults.

#### Install Vertex SDK for Python

In [3]:
!pip -q install {USER_FLAG} --upgrade kfp
!pip -q install {USER_FLAG} --upgrade google-cloud-pipeline-components 
!pip -q install {USER_FLAG} --upgrade google-cloud-aiplatform

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [4]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

Check the versions of the packages you installed.  The KFP SDK version should be >=1.6.

In [1]:
!python3 -c "import kfp; print('kfp version: {}'.format(kfp.__version__))"
!python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

kfp version: 1.8.11
google_cloud_pipeline_components version: 0.2.2


## Before you begin

This notebook does not require a GPU runtime.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component). 
Also [enable the Cloud Build API](https://console.cloud.google.com/flows/enableapi?apiid=cloudbuild.googleapis.com).

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [2]:
import os
# Get your Google Cloud project ID from gcloud
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null

try:
    PROJECT_ID = shell_output[0]
except IndexError:
    PROJECT_ID = None

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  laah-play


Otherwise, set your project ID here.

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "python-docs-samples-tests"  # @param {type:"string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [3]:
from datetime import datetime

def get_timestamp():
    return datetime.now().strftime("%Y%m%d%H%M%S")

TIMESTAMP = get_timestamp()
print(f"TIMESTAMP = {TIMESTAMP}")

TIMESTAMP = 20220126195815


### Authenticate your Google Cloud account

**If you are using AI Platform Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "AI Platform"
into the filter box, and select
   **AI Platform Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on AI Platform, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket as necessary

You will need a Cloud Storage bucket for this example.  If you don't have one that you want to use, you can make one now.


Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where AI Platform (Unified) services are
available](https://cloud.google.com/ai-platform-unified/docs/general/locations#available_regions). You may
not use a Multi-Regional Storage bucket for training with AI Platform.

**Change the bucket name below** before running the next cell.

In [4]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}
#BUCKET_NAME = "gs://laah-pipeline-aip-20220124194618"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

In [5]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "-aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [6]:
! gsutil mb -l $REGION $BUCKET_NAME

Creating gs://laah-play-aip-20220126195815/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [7]:
! gsutil ls -al $BUCKET_NAME

### Import libraries and define constants



Define some constants. See the "Before you begin" section of the Managed Pipelines User Guide for information on creating your API key.


In [8]:
APP_NAME = "custom-rf-classifier"

In [9]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

PIPELINE_ROOT = f'{BUCKET_NAME}/pipeline_root/{APP_NAME}'

PIPELINE_ROOT

env: PATH=/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


'gs://laah-play-aip-20220126195815/pipeline_root/custom-rf-classifier'

Do some imports:

In [10]:
import json
from typing import NamedTuple, List

from google_cloud_pipeline_components import aiplatform as aip_components
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs

from kfp.v2 import compiler
from kfp.v2 import dsl
from kfp.v2.dsl import (
    component,
    InputPath,
    OutputPath,
    Input,
    Output,
    Artifact,
    Dataset,
    Model,
    ClassificationMetrics,
    Metrics,
)
from kfp.v2.google.client import AIPlatformClient

## Define the pipeline components



In [22]:
!mkdir ./pipelines
!mkdir ./trainer
!mkdir ./predictor

mkdir: cannot create directory ‘./pipelines’: File exists
mkdir: cannot create directory ‘./trainer’: File exists


### 1. Component to create custom training container image

In [14]:
%%bash

cat << EOF > ./trainer/Dockerfile

FROM gcr.io/deeplearning-platform-release/r-cpu

# install dependencies
RUN apt-get -y update && apt-get -y upgrade && apt-get -y install gfortran

# Installs R Dependencies
RUN mkdir /root/trainer && mkdir /root/trainer/libs

# Copies the trainer code
COPY train.R /root/trainer/

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["Rscript", "/root/trainer/train.R"]

EOF

In [29]:
%%bash

cat << EOF > ./trainer/train.R

options(repos = "http://cran.us.r-project.org")
.libPaths("/root/trainer/libs")
# Install required libraries
install.packages("randomForest", quiet=TRUE)

library(randomForest)


# Get training data

system(paste0("gsutil cp ", "gs://laah-play-aip-20220126195815/iris.csv", " ./tmp/train_data.csv"))
iris<- read.csv("./tmp/train_data.csv")

# train model

model = randomForest(as.factor(Species) ~ ., data = iris)
# save model
#save(model, file = "./predictor/model.RData")

# Export models to Google Cloud Storage

saveRDS(model, './tmp/model.rds')
system(paste0("gsutil cp ./tmp/model.rds ", "gs://laah-play-aip-20220126195815/custom-rf-classifier/serve/predictor", "/model.rds"))



EOF

In [16]:
print(f"{BUCKET_NAME}/{APP_NAME}/train/")

gs://laah-play-aip-20220126195815/custom-rf-classifier/train/


In [47]:
!gsutil cp ./trainer/Dockerfile ./trainer/train.R {BUCKET_NAME}/{APP_NAME}/train/

Copying file://./trainer/Dockerfile [Content-Type=application/octet-stream]...
Copying file://./trainer/train.R [Content-Type=application/octet-stream]...     
/ [2 files][  954.0 B/  954.0 B]                                                
Operation completed over 2 objects/954.0 B.                                      


In [18]:
!gsutil ls -r {BUCKET_NAME}/{APP_NAME}/train

gs://laah-play-aip-20220126195815/custom-rf-classifier/train/:
gs://laah-play-aip-20220126195815/custom-rf-classifier/train/Dockerfile
gs://laah-play-aip-20220126195815/custom-rf-classifier/train/train.R


In [19]:
@component(
    #base_image="gcr.io/google.com/cloudsdktool/cloud-sdk:latest",
    base_image="python:3.7",
    packages_to_install=["google-cloud-build"],
    output_component_file="./pipelines/build_custom_train_image.yaml",
)
def build_custom_train_image(
    project: str,
    gs_train_src_path: str,
    model_display_name: str
) -> NamedTuple(
    "Outputs",
    [
        ("training_container_uri", str), 
    ]
):

    from datetime import datetime
    import logging
    import os

    from google.cloud.devtools import cloudbuild_v1 as cloudbuild

    logging.getLogger().setLevel(logging.INFO)
    build_client = cloudbuild.services.cloud_build.CloudBuildClient()

    gs_dockerfile_path = os.path.join(gs_train_src_path, 'Dockerfile')
    gs_train_src_path = os.path.join(gs_train_src_path, 'train.R')

    build_version = datetime.now().strftime("%Y%m%d%H%M%S")
    image_name = f"r_train_{model_display_name}"
    training_image_uri = f"gcr.io/{project}/{image_name}:{build_version}"
    logging.info(f"training_image_uri: {training_image_uri}")

    build = cloudbuild.Build(
        images=[training_image_uri]
    )
    build.steps = [
        {
            "name": "gcr.io/cloud-builders/gsutil",
            "args": ["cp", "-r", gs_train_src_path, "."]
        },
        {
            "name": "gcr.io/cloud-builders/gsutil",
            "args": ["cp", gs_dockerfile_path, "Dockerfile"]
        },
        {
            "name": "gcr.io/cloud-builders/gcloud",
            "entrypoint": "bash",
            "args": ["-c", "ls -ltr"]
        },
        {
            "name": 'gcr.io/cloud-builders/docker',
            "args": [ 'build', '-t', training_image_uri, '.' ]
        }
    ]
    operation = build_client.create_build(project_id=project, build=build, timeout=1800)
    logging.info("IN PROGRESS:")
    logging.info(operation.metadata)

    result = operation.result()
    # Print the completed status
    logging.info("RESULT:", result.status)
    return(training_image_uri,)

### 2. Component to run training job on Vertex AI

In [20]:
@component(
    # base_image="gcr.io/google.com/cloudsdktool/cloud-sdk:latest", 
    base_image="python:3.7",
    packages_to_install=["google-cloud-aiplatform", "pandas", "fsspec"],
    output_component_file="./pipelines/submit_custom_training_job.yaml")
def submit_custom_training_job( 
    project: str,
    bucket: str,
    training_container_image_uri: str,
    metrics: Output[Metrics],
    model: Output[Model],
    eval_metric_key: str,
    model_display_name: str
):

    from google.cloud import aiplatform
    from datetime import datetime
    import logging

    import pandas as pd

    TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

    logging.info(f"Model artifacts located at model.uri = {model.uri}")
    logging.info(f"Model artifacts located at model.path = {model.path}")

    # initialize vertex sdk
    aiplatform.init(
        project=project, 
        staging_bucket=bucket
    )

    JOB_NAME = f"{model_display_name}-r-cstm-cntr-{TIMESTAMP}"

    # configure the job with container image spec
    job = aiplatform.CustomContainerTrainingJob(
        display_name=JOB_NAME, 
        container_uri=training_container_image_uri)

    # define worker pool specs
    train_replica_count = 1
    train_machine_type = "n1-standard-4"

    # output directories
    # base_output_dir = f"{bucket}/{JOB_NAME}"

    # submit the custom job to Vertex training service
    job_response = job.run(
        replica_count=train_replica_count,
        machine_type=train_machine_type,
        base_output_dir=model.uri
    )

    start, end = job._gca_resource.start_time, job._gca_resource.end_time

    logging.info(f"Model artifacts located at {model.uri}/model/{model_display_name}")
    model.metadata["framework"] = "r"

    logging.info(f"Model artifacts located at model.uri = {model.uri}")
    logging.info(f"Model artifacts located at model.path = {model.path}")
    model.metadata["time_to_train_in_seconds"] = (end - start).total_seconds()  

### 3. Component to create serving container image

In [26]:
%%bash

cat << EOF > ./predictor/serving.R

# serving.R

library("randomForest")

#* Health check
#* @get /ping
#* @serializer unboxedJSON
function() {
    list(status = "OK")
}

#* @apiTitle flower classifier
#* @param petal_length 
#* @param petal_width 
#* @param sepal_length
#* @param sepal_width
#* @post /classify
function (req) 
{
    instances <- as.data.frame(jsonlite::fromJSON(req\$postBody))
    results <- list()
    
    load("./app/model.rds")
    
    for(i in 1:nrow(instances)) {       # for-loop over columns
        petal_length <- instances[i, "instances.petal_length"]
        petal_width <- instances[i, "instances.petal_width"]
        sepal_length <- instances[i, "instances.sepal_length"]
        sepal_width <- instances[i, "instances.sepal_width"]
        test = c(sepal_length, sepal_width, petal_length, petal_width)
        test = sapply(test, as.numeric)
        test = data.frame(matrix(test, ncol = 4))
        colnames(test) = colnames(iris[, 1:4])
        results <- append(results, predict(model, test))
    }
    
    list(predictions = results)
}

EOF

In [24]:
%%bash

cat << EOF > ./predictor/startServer.R

library(plumber)
pr <- plumb("serving.R")
pr\$run(host = "0.0.0.0", port = 7080)

EOF

In [25]:
%%bash -s $APP_NAME

APP_NAME=$1

cat << EOF > ./predictor/Dockerfile

FROM rstudio/plumber

# install random forest
RUN R -e 'install.packages(c("randomForest"), repos = "https://cran.rstudio.com/")'

# Copy model and script
RUN mkdir /app
COPY model.rds /app
COPY serving.R /app
COPY startServer.R /app
WORKDIR /app

# plumber & run server
EXPOSE 7080

ENTRYPOINT ["R", "-f", "/app/startServer.R"]

EOF

echo "Writing ./predictor/Dockerfile"

Writing ./predictor/Dockerfile


In [27]:
!gsutil cp -r ./predictor/ {BUCKET_NAME}/{APP_NAME}/serve/

Copying file://./predictor/Dockerfile [Content-Type=application/octet-stream]...
Copying file://./predictor/startServer.R [Content-Type=application/octet-stream]...
Copying file://./predictor/serving.R [Content-Type=application/octet-stream]... 
/ [3 files][  1.4 KiB/  1.4 KiB]                                                
Operation completed over 3 objects/1.4 KiB.                                      


In [28]:
!gsutil ls -lR {BUCKET_NAME}/{APP_NAME}/serve/

gs://laah-play-aip-20220126195815/custom-rf-classifier/serve/:

gs://laah-play-aip-20220126195815/custom-rf-classifier/serve/predictor/:
       331  2022-01-26T20:03:35Z  gs://laah-play-aip-20220126195815/custom-rf-classifier/serve/predictor/Dockerfile
      1044  2022-01-26T20:03:35Z  gs://laah-play-aip-20220126195815/custom-rf-classifier/serve/predictor/serving.R
        82  2022-01-26T20:03:35Z  gs://laah-play-aip-20220126195815/custom-rf-classifier/serve/predictor/startServer.R
TOTAL: 3 objects, 1457 bytes (1.42 KiB)


In [31]:
@component(
    #base_image="gcr.io/google.com/cloudsdktool/cloud-sdk:latest",
    base_image="python:3.7",
    packages_to_install=["google-cloud-build"],
    output_component_file="./pipelines/build_custom_serving_image.yaml",
)
def build_custom_serving_image(
    project: str,
    gs_model_artifacts: Input[Model],
    gs_serving_dependencies_path: str,
    model_display_name: str
)-> NamedTuple(
    "Outputs",
    [
        ("serving_container_uri", str)
    ],
):

    from datetime import datetime
    from collections import namedtuple
    import logging
    import os

    from google.cloud.devtools import cloudbuild_v1 as cloudbuild

    logging.getLogger().setLevel(logging.INFO)
    build_client = cloudbuild.services.cloud_build.CloudBuildClient()

    logging.info(f"gs_serving_dependencies_path: {gs_serving_dependencies_path}")
    logging.info(f"gs_model_artifacts.uri: {gs_model_artifacts.uri}")
    logging.info(f"gs_model_artifacts.path: {gs_model_artifacts.path}")

    gs_dockerfile_path = os.path.join(gs_serving_dependencies_path, 'predictor/Dockerfile')
    gs_predictor_src_path = os.path.join(gs_serving_dependencies_path)

    build_version = datetime.now().strftime("%Y%m%d%H%M%S")
    image_name = f"r_predict_{model_display_name}"
    serving_image_uri = f"gcr.io/{project}/{image_name}:{build_version}"
    logging.info(f"serving_image_uri: {serving_image_uri}")

    build = cloudbuild.Build(
        images=[serving_image_uri]
    )
    build.steps = [
#         {
#             "name": "gcr.io/cloud-builders/gcloud",
#             "entrypoint": "bash",
#             "args": ["-c", "mkdir model"]
#         },
#         {
#             "name": "gcr.io/cloud-builders/gsutil",
#             "args": ["cp", f"{gs_model_artifacts.uri}/model/model.RData", "model.RData"]
#         },
        {
            "name": "gcr.io/cloud-builders/gsutil",
            "args": ["cp", "-r", f"{gs_serving_dependencies_path}", "."]
        },
        {
            "name": "gcr.io/cloud-builders/gcloud",
            "entrypoint": "bash",
            "args": ["-c", "ls -ltrR"]
        },
        {
          "name": 'gcr.io/cloud-builders/docker',
          "args": [ 'build', '-t', serving_image_uri, './predictor' ]
        }
    ]
    operation = build_client.create_build(project_id=project, build=build, timeout=1800)
    logging.info("IN PROGRESS:")
    logging.info(operation.metadata)

    result = operation.result()
    # Print the completed status
    logging.info("RESULT:", result.status)

    outputs = namedtuple("Outputs", ["serving_container_uri"])

    return outputs(serving_image_uri)

### 4. Component to test model deployment making online prediction requests

In [32]:
@component(
    #base_image="gcr.io/google.com/cloudsdktool/cloud-sdk:latest",
    base_image="python:3.7",
    packages_to_install=["google-cloud-build", "google-cloud-aiplatform"],
    output_component_file="./pipelines/make_prediction_request.yaml")
def make_prediction_request( 
    project: str,
    bucket: str,
    endpoint: str,
    instances: List
    ):
    from datetime import datetime
    import logging
    import base64
    import ast

    from google.cloud import aiplatform

    logging.getLogger().setLevel(logging.INFO)
    aiplatform.init(project=project, staging_bucket=bucket)
    logging.info(f"Endpoint: {endpoint}")
    endpoint = ast.literal_eval(endpoint)
    endpoint_uri = endpoint["resources"][0]["resourceUri"]
    logging.info(f"Endpoint URI: {endpoint_uri}")
    endpoint_name = "/".join(endpoint_uri.split("/")[4:-2])
    logging.info(f"Endpoint Name: {endpoint_name}")
    _endpoint = aiplatform.Endpoint(endpoint_name)

    response = _endpoint.predict(instances)
    logging.info(f"Prediction response: {response}")

In [50]:
ENDPOINT_ID="8134283779310092288"
PROJECT_ID="307074083428"
INPUT_DATA_FILE="INPUT-JSON"

projects/307074083428/locations/us-central1/endpoints/8246873769994354688


In [ ]:
from google.cloud import aiplatform

endpoint_name= "projects/307074083428/locations/us-central1/endpoints/5134283779310092288"

aiplatform.init(project=PROJECT_ID, location=REGION)
endpoint = aiplatform.Endpoint(endpoint_name)

print(endpoint._gca_resource.name)

instances = [
    {"sepal_width": 1, "sepal_length": 2, "petal_width": 3, "petal_length": 1},
    {"sepal_width": 4, "sepal_length": 2, "petal_width": 1, "petal_length": 1}
]
response = endpoint.predict(instances)

## Define Pipeline

In [33]:
@dsl.pipeline(
    name="r-flower-classifier-pipeline",
    pipeline_root=PIPELINE_ROOT,
)
def r_flower_classifier_pipeline(
    project: str,
    bucket: str,
    region: str,
    gs_train_script_path: str,
    gs_serving_dependencies_path: str,
    model_display_name: str,
    serving_container_health_route: str,
    serving_container_predict_route: str,
    serving_container_ports: list,
    eval_acc_threshold: float,
    pipeline_job_id: str,
    pipeline_name: str
):
    # build custom container for training job passing the 
    # GCS location of the training application code 
    build_custom_train_image_task = build_custom_train_image(
        project=project,
        gs_train_src_path=gs_train_script_path,
        model_display_name=model_display_name
    ).set_caching_options(True).set_display_name("Build custom training image")

    # train the model on Vertex AI by submitting a CustomJob 
    # using the custom container
    training_container_image_uri = build_custom_train_image_task.outputs["training_container_uri"]
    run_train_task = submit_custom_training_job(
        project=project,
        bucket=bucket,
        training_container_image_uri=training_container_image_uri,
        eval_metric_key="eval_accuracy",
        model_display_name=model_display_name
    ).set_caching_options(True).set_display_name("Run custom training job")

    # build custom container for serving predictions using 
    # the trained model artifacts served by TorchServe
    build_custom_serving_image_task = build_custom_serving_image(
        project=project,
        gs_model_artifacts=run_train_task.outputs["model"],
        gs_serving_dependencies_path=gs_serving_dependencies_path,
        model_display_name=model_display_name
    ).set_caching_options(True).set_display_name("Build custom serving image")

    # upload model to vertex ai
    # NOTE: model artifacts and the prediction handler are part of the container
    serving_container_image_uri = build_custom_serving_image_task.outputs['serving_container_uri']
    model_upload_task = aip_components.ModelUploadOp(
        project=project, 
        display_name=model_display_name,
        serving_container_image_uri=serving_container_image_uri,
        serving_container_predict_route=serving_container_predict_route,
        serving_container_health_route=serving_container_health_route,
        serving_container_ports=serving_container_ports
    ).set_caching_options(True).set_display_name("Upload model")

    # create endpoint to deploy one or more models
    # An endpoint provides a service URL where the prediction requests are sent
    endpoint_create_task = aip_components.EndpointCreateOp(
        project=project,
        display_name=model_display_name,
    ).set_caching_options(True).set_display_name("Create endpoint")

    # deploy models to endpoint to associates physical resources with the model 
    # so it can serve online predictions 
    model_deploy_task = aip_components.ModelDeployOp(
        endpoint=endpoint_create_task.outputs["endpoint"],
        model=model_upload_task.outputs["model"],
        deployed_model_display_name=model_display_name,
        dedicated_resources_machine_type="n1-standard-4",
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
        traffic_split='{ "0": 100 }'
    ).set_caching_options(True).set_display_name("Deploy model to endpoint")

    # test model deployment by making online prediction requests
#     test_instances = [
#         {"sepal_width": 1, "sepal_length": 2, "petal_width": 3, "petal_length": 1},
#         {"sepal_width": 4, "sepal_length": 2, "petal_width": 1, "petal_length": 1}
#     ]
#     predict_test_instances_task = make_prediction_request(
#         project,
#         bucket,
#         model_deploy_task.outputs["gcp_resources"],
#         test_instances
#     ).set_caching_options(True).set_display_name("Test model deployment making online predictions")


In [34]:
PIPELINE_JSON_SPEC_PATH = './pipelines/r_flower_classifier_pipeline_spec.json'
compiler.Compiler().compile(
    pipeline_func=r_flower_classifier_pipeline,
    package_path=PIPELINE_JSON_SPEC_PATH)

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1266: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


In [35]:
aiplatform.init(project=PROJECT_ID, location=REGION)

In [51]:
PIPELINE_NAME = f"r-flower-classifier-pipeline"
PIPELINE_JOB_ID = f"pipeline-{APP_NAME}-{get_timestamp()}"
TRAIN_APP_CODE_PATH = f"{BUCKET_NAME}/{APP_NAME}/train/"
SERVE_DEPENDENCIES_PATH = f"{BUCKET_NAME}/{APP_NAME}/serve/predictor"
pipeline_params = {
    "gs_train_script_path": TRAIN_APP_CODE_PATH,
    "gs_serving_dependencies_path": SERVE_DEPENDENCIES_PATH,
    "project": PROJECT_ID,
    "bucket": BUCKET_NAME,
    "region": REGION,
    "model_display_name": APP_NAME,
    "serving_container_health_route": "/ping",
    "serving_container_predict_route": f"/predictions/{APP_NAME}",
    "serving_container_ports": [{"containerPort" : 7080}],
    "eval_acc_threshold": 0.88,
    "pipeline_name": PIPELINE_NAME,
    "pipeline_job_id": PIPELINE_JOB_ID
}

In [52]:
pipeline_job = pipeline_jobs.PipelineJob(
    display_name=PIPELINE_NAME,
    job_id=PIPELINE_JOB_ID,
    template_path=PIPELINE_JSON_SPEC_PATH,
    pipeline_root=PIPELINE_ROOT,
    parameter_values=pipeline_params,
    enable_caching=True
)

In [53]:
response = pipeline_job.run(sync=False)
response

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/959641146622/locations/us-central1/pipelineJobs/pipeline-custom-rf-classifier-20220126235442
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/959641146622/locations/us-central1/pipelineJobs/pipeline-custom-rf-classifier-20220126235442')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/pipeline-custom-rf-classifier-20220126235442?project=959641146622
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/959641146622/locations/us-central1/pipelineJobs/pipeline-custom-rf-classifier-20220126235442 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projec

In [48]:
df = aiplatform.get_pipeline_df(pipeline=PIPELINE_NAME.replace("_", "-"))
df

,pipeline_name,run_name,param.input:region,param.input:bucket,param.input:gs_serving_dependencies_path,param.input:serving_container_ports,param.input:pipeline_job_id,param.input:serving_container_predict_route,param.input:model_display_name,param.input:gs_train_script_path,param.input:project,param.input:serving_container_health_route,param.input:pipeline_name,param.input:eval_acc_threshold
0,r-flower-classifier-pipeline,pipeline-custom-rf-classifier-20220126164728,us-central1,gs://laah-pipeline-aip-20220126144741,gs://laah-pipeline-aip-20220126144741/custom-r...,"[{""containerPort"": 7080}]",pipeline-custom-rf-classifier-20220126164728,/predictions/custom-rf-classifier,custom-rf-classifier,gs://laah-pipeline-aip-20220126144741/custom-r...,laah-pipeline,/ping,r-flower-classifier-pipeline,0.88
1,r-flower-classifier-pipeline,pipeline-custom-rf-classifier-20220126164246,us-central1,gs://laah-pipeline-aip-20220126144741,gs://laah-pipeline-aip-20220126144741/custom-r...,"[{""containerPort"": 7080}]",pipeline-custom-rf-classifier-20220126164246,/predictions/custom-rf-classifier,custom-rf-classifier,gs://laah-pipeline-aip-20220126144741/custom-r...,laah-pipeline,/ping,r-flower-classifier-pipeline,0.88
2,r-flower-classifier-pipeline,pipeline-custom-rf-classifier-20220126154159,us-central1,gs://laah-pipeline-aip-20220126144741,gs://laah-pipeline-aip-20220126144741/custom-r...,"[{""containerPort"": 7080}]",pipeline-custom-rf-classifier-20220126154159,/predictions/custom-rf-classifier,custom-rf-classifier,gs://laah-pipeline-aip-20220126144741/custom-r...,laah-pipeline,/ping,r-flower-classifier-pipeline,0.88
3,r-flower-classifier-pipeline,pipeline-custom-rf-classifier-20220126151414,us-central1,gs://laah-pipeline-aip-20220126144741,gs://laah-pipeline-aip-20220126144741/custom-r...,"[{""containerPort"": 7080}]",pipeline-custom-rf-classifier-20220126151414,/predictions/custom-rf-classifier,custom-rf-classifier,gs://laah-pipeline-aip-20220126144741/custom-r...,laah-pipeline,/ping,r-flower-classifier-pipeline,0.88
4,r-flower-classifier-pipeline,pipeline-custom-rf-classifier-20220125223607,us-central1,gs://laah-pipeline-aip-20220125173552,gs://laah-pipeline-aip-20220125173552/custom-r...,"[{""containerPort"": 7080}]",pipeline-custom-rf-classifier-20220125223607,/predictions/custom-rf-classifier,custom-rf-classifier,gs://laah-pipeline-aip-20220125173552/custom-r...,laah-pipeline,/ping,r-flower-classifier-pipeline,0.88
5,r-flower-classifier-pipeline,pipeline-custom-rf-classifier-20220125221626,us-central1,gs://laah-pipeline-aip-20220125173552,gs://laah-pipeline-aip-20220125173552/custom-r...,"[{""containerPort"": 7080}]",pipeline-custom-rf-classifier-20220125221626,/predictions/custom-rf-classifier,custom-rf-classifier,gs://laah-pipeline-aip-20220125173552/custom-r...,laah-pipeline,/ping,r-flower-classifier-pipeline,0.88
6,r-flower-classifier-pipeline,pipeline-custom-rf-classifier-20220125220543,us-central1,gs://laah-pipeline-aip-20220125173552,gs://laah-pipeline-aip-20220125173552/custom-r...,"[{""containerPort"": 7080}]",pipeline-custom-rf-classifier-20220125220543,/predictions/custom-rf-classifier,custom-rf-classifier,gs://laah-pipeline-aip-20220125173552/custom-r...,laah-pipeline,/ping,r-flower-classifier-pipeline,0.88
7,r-flower-classifier-pipeline,pipeline-custom-rf-classifier-20220125184833,us-central1,gs://laah-pipeline-aip-20220125173552,gs://laah-pipeline-aip-20220125173552/custom-r...,"[{""containerPort"": 7080}]",pipeline-custom-rf-classifier-20220125184833,/predictions/custom-rf-classifier,custom-rf-classifier,gs://laah-pipeline-aip-20220125173552/custom-r...,laah-pipeline,/ping,r-flower-classifier-pipeline,0.88
8,r-flower-classifier-pipeline,pipeline-custom-rf-classifier-20220125175224,us-central1,gs://laah-pipeline-aip-20220125173552,gs://laah-pipeline-aip-20220125173552/custom-r...,"[{""containerPort"": 7080}]",pipeline-custom-rf-classifier-20220125175224,/predictions/custom-rf-classifier,custom-rf-classifier,gs://laah-pipeline-